If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [ ]:
#./tint-runner-0.2-bin/tint/tint.sh -i input.txt -o output.txt -f json --properties annotators=tokenize,ssplit

In [34]:
import os
from subprocess import Popen, PIPE, STDOUT
import pandas as pd

tint_path = '/mnt/d/school/multicardioner/tint-runner-0.2-bin/tint'
text_path = '/mnt/d/school/multicardioner/multicardioner_train+dev+test+bg_240502/test+background/it/output'
annotations_path = '' #'/mnt/d/school/multicardioner/multicardioner_train+dev+test+bg_240502/track2/drugtemist_train/it/brat'
output_path = '/mnt/d/school/multicardioner/multicardioner_train+dev+test+bg_240502/test+background/it/sentences' 
sents_per_batch = 1 # max number of sentences in each output file
is_test = True

In [35]:
import json
from pathlib import Path

sentences = []
for file_name in os.listdir(text_path):
    if '.txt' in file_name: # and 'casos_clinicos_cardiologia129' in file_name:
        file_name_no_ext = file_name.rstrip('.txt')
                 
        file_path = Path(f'{text_path}/{file_name}')
        full_text = file_path.read_text()

        with open(f'{text_path}/{file_name}') as file:
            file_json = json.load(file)
            
        #print(len(file_json['sentences']))
        
        print("Processing: ", file_name_no_ext)
        #T105	FARMACO 911 922	azatioprina

        if not is_test:
            annotations = pd.read_csv(f'{annotations_path}/{file_name_no_ext}.ann', sep='\t', 
                                      names=['ann_type', 'entity_type', 'text'], encoding='utf-8', keep_default_na=False)

            # drop annotator comments
            annotations.drop(annotations[annotations['ann_type'].str.startswith('#')].index, inplace=True)

            annotations['start_pos'] = annotations['entity_type'].apply(lambda v: int(v.split()[1]))
            annotations['end_pos'] = annotations['entity_type'].apply(lambda v: int(v.split()[2]))
            annotations['entity_type'] = annotations['entity_type'].apply(lambda v: v.split()[0])
            annotations.sort_values(by=['start_pos'], inplace=True, ignore_index=True)
        else:
            annotations = pd.DataFrame(columns=['ann_type', 'text'])
            
        total_chars = 0
        annotation_index = 0
        total_annotations = len(annotations)
        line_number = 0
        batch = []
        batch_number = 0
        for line in file_json['sentences']:
            line_number += 1
            batch.append(line['text'])

            if line_number % sents_per_batch == 0:
                batch_number += 1
                current_batch_annotation_indecies = []
                text = ''.join(batch)
                #max_index = total_chars + len(text) # - 1;
                #max_index = total_chars + full_text[total_chars:].find(text) + len(text) - 1
                batch_start = line['characterOffsetBegin'] #total_chars + full_text[total_chars:].find(text.strip()) #+ len(text) - 1
                batch_end = line['characterOffsetEnd'] # batch_start + len(text) - 1
                #print(text, batch_start, batch_end)
                if batch_start < 0:
                    print('batch_start', full_text[total_chars:], text, total_chars)
                
                while annotation_index < total_annotations and annotations.iloc[annotation_index]['start_pos'] >= batch_start and annotations.iloc[annotation_index]['start_pos'] < batch_end:
                    annotation_text = str(annotations.iloc[annotation_index]['text'])
                    #print(annotation_text)
                    #new_start_pos = text.find(annotation_text)
                    new_start_pos = annotations.iloc[annotation_index]['start_pos'] - batch_start
                    
                    if new_start_pos < 0:
                        print('new_start_pos', text, annotation_text, batch_number, max_index)
                        
                    annotations.at[annotation_index, 'start_pos'] = new_start_pos
                    annotations.at[annotation_index, 'end_pos'] = new_start_pos + len(annotation_text)
                    current_batch_annotation_indecies.append(annotation_index)
                    annotation_index += 1

                format_batch_number = str(batch_number).zfill(3)
                with open(f'{output_path}/splits_{sents_per_batch}/{file_name_no_ext}-b-{format_batch_number}-{batch_start}.txt', 'w', encoding='utf-8') as out_file:
                    out_file.write(text)

                # file_name_no_ext, batch_number, batch_start, text
                sentences.append({
                    'filename': file_name_no_ext,
                    'batch_number': batch_number,
                    'batch_start': batch_start,
                    'text': text
                })
                
                current_batch_annotations = annotations.iloc[current_batch_annotation_indecies,].copy().reset_index(drop=True)
                current_batch_annotations['entity_type_with_positions'] = None
                if not current_batch_annotations.empty:
                    current_batch_annotations['row_name'] = current_batch_annotations.apply(lambda row: row.name + 1, axis=1)
                    current_batch_annotations['ann_type'] = current_batch_annotations.apply(lambda row: f'{row["ann_type"].rstrip("0123456789")}{row.name + 1}', axis=1)
                    current_batch_annotations['entity_type_with_positions'] = current_batch_annotations.apply(lambda row: f'{row["entity_type"]} {row["start_pos"]} {row["end_pos"]}', axis=1)
                    current_batch_annotations.drop(columns=['start_pos', 'end_pos', 'entity_type'], inplace=True)

                current_batch_annotations.to_csv(f'{output_path}/splits_{sents_per_batch}/{file_name_no_ext}-b-{format_batch_number}-{batch_start}.ann', sep='\t', header=False, index=False, columns=['ann_type', 'entity_type_with_positions', 'text'], encoding='utf-8')

                total_chars = batch_end #len(text) # -1
                batch.clear()
                
        if annotation_index < total_annotations:
            print('missing annotations')

Processing:  multicardioner_test+bg_1
Processing:  multicardioner_test+bg_10
Processing:  multicardioner_test+bg_100
Processing:  multicardioner_test+bg_1000
Processing:  multicardioner_test+bg_1001
Processing:  multicardioner_test+bg_1002
Processing:  multicardioner_test+bg_1003
Processing:  multicardioner_test+bg_1004
Processing:  multicardioner_test+bg_1005
Processing:  multicardioner_test+bg_1006
Processing:  multicardioner_test+bg_1007
Processing:  multicardioner_test+bg_1008
Processing:  multicardioner_test+bg_1009
Processing:  multicardioner_test+bg_101
Processing:  multicardioner_test+bg_1010
Processing:  multicardioner_test+bg_1011
Processing:  multicardioner_test+bg_1012
Processing:  multicardioner_test+bg_1013
Processing:  multicardioner_test+bg_1014
Processing:  multicardioner_test+bg_1015
Processing:  multicardioner_test+bg_1016
Processing:  multicardioner_test+bg_1017
Processing:  multicardioner_test+bg_1018
Processing:  multicardioner_test+bg_1019
Processing:  multicardi

Processing:  multicardioner_test+bg_118
Processing:  multicardioner_test+bg_1180
Processing:  multicardioner_test+bg_1181
Processing:  multicardioner_test+bg_1182
Processing:  multicardioner_test+bg_1183
Processing:  multicardioner_test+bg_1184
Processing:  multicardioner_test+bg_1185
Processing:  multicardioner_test+bg_1186
Processing:  multicardioner_test+bg_1187
Processing:  multicardioner_test+bg_1188
Processing:  multicardioner_test+bg_1189
Processing:  multicardioner_test+bg_119
Processing:  multicardioner_test+bg_1190
Processing:  multicardioner_test+bg_1191
Processing:  multicardioner_test+bg_1192
Processing:  multicardioner_test+bg_1193
Processing:  multicardioner_test+bg_1194
Processing:  multicardioner_test+bg_1195
Processing:  multicardioner_test+bg_1196
Processing:  multicardioner_test+bg_1197
Processing:  multicardioner_test+bg_1198
Processing:  multicardioner_test+bg_1199
Processing:  multicardioner_test+bg_12
Processing:  multicardioner_test+bg_120
Processing:  multicar

Processing:  multicardioner_test+bg_1361
Processing:  multicardioner_test+bg_1362
Processing:  multicardioner_test+bg_1363
Processing:  multicardioner_test+bg_1364
Processing:  multicardioner_test+bg_1365
Processing:  multicardioner_test+bg_1366
Processing:  multicardioner_test+bg_1367
Processing:  multicardioner_test+bg_1368
Processing:  multicardioner_test+bg_1369
Processing:  multicardioner_test+bg_137
Processing:  multicardioner_test+bg_1370
Processing:  multicardioner_test+bg_1371
Processing:  multicardioner_test+bg_1372
Processing:  multicardioner_test+bg_1373
Processing:  multicardioner_test+bg_1374
Processing:  multicardioner_test+bg_1375
Processing:  multicardioner_test+bg_1376
Processing:  multicardioner_test+bg_1377
Processing:  multicardioner_test+bg_1378
Processing:  multicardioner_test+bg_1379
Processing:  multicardioner_test+bg_138
Processing:  multicardioner_test+bg_1380
Processing:  multicardioner_test+bg_1381
Processing:  multicardioner_test+bg_1382
Processing:  multi

Processing:  multicardioner_test+bg_1543
Processing:  multicardioner_test+bg_1544
Processing:  multicardioner_test+bg_1545
Processing:  multicardioner_test+bg_1546
Processing:  multicardioner_test+bg_1547
Processing:  multicardioner_test+bg_1548
Processing:  multicardioner_test+bg_1549
Processing:  multicardioner_test+bg_155
Processing:  multicardioner_test+bg_1550
Processing:  multicardioner_test+bg_1551
Processing:  multicardioner_test+bg_1552
Processing:  multicardioner_test+bg_1553
Processing:  multicardioner_test+bg_1554
Processing:  multicardioner_test+bg_1555
Processing:  multicardioner_test+bg_1556
Processing:  multicardioner_test+bg_1557
Processing:  multicardioner_test+bg_1558
Processing:  multicardioner_test+bg_1559
Processing:  multicardioner_test+bg_156
Processing:  multicardioner_test+bg_1560
Processing:  multicardioner_test+bg_1561
Processing:  multicardioner_test+bg_1562
Processing:  multicardioner_test+bg_1563
Processing:  multicardioner_test+bg_1564
Processing:  multi

Processing:  multicardioner_test+bg_1726
Processing:  multicardioner_test+bg_1727
Processing:  multicardioner_test+bg_1728
Processing:  multicardioner_test+bg_1729
Processing:  multicardioner_test+bg_173
Processing:  multicardioner_test+bg_1730
Processing:  multicardioner_test+bg_1731
Processing:  multicardioner_test+bg_1732
Processing:  multicardioner_test+bg_1733
Processing:  multicardioner_test+bg_1734
Processing:  multicardioner_test+bg_1735
Processing:  multicardioner_test+bg_1736
Processing:  multicardioner_test+bg_1737
Processing:  multicardioner_test+bg_1738
Processing:  multicardioner_test+bg_1739
Processing:  multicardioner_test+bg_174
Processing:  multicardioner_test+bg_1740
Processing:  multicardioner_test+bg_1741
Processing:  multicardioner_test+bg_1742
Processing:  multicardioner_test+bg_1743
Processing:  multicardioner_test+bg_1744
Processing:  multicardioner_test+bg_1745
Processing:  multicardioner_test+bg_1746
Processing:  multicardioner_test+bg_1747
Processing:  multi

...

Processing:  multicardioner_test+bg_7178
Processing:  multicardioner_test+bg_7179
Processing:  multicardioner_test+bg_718
Processing:  multicardioner_test+bg_7180
Processing:  multicardioner_test+bg_7181
Processing:  multicardioner_test+bg_7182
Processing:  multicardioner_test+bg_7183
Processing:  multicardioner_test+bg_7184
Processing:  multicardioner_test+bg_7185
Processing:  multicardioner_test+bg_7186
Processing:  multicardioner_test+bg_7187
Processing:  multicardioner_test+bg_7188
Processing:  multicardioner_test+bg_7189
Processing:  multicardioner_test+bg_719
Processing:  multicardioner_test+bg_7190
Processing:  multicardioner_test+bg_7191
Processing:  multicardioner_test+bg_7192
Processing:  multicardioner_test+bg_7193
Processing:  multicardioner_test+bg_7194
Processing:  multicardioner_test+bg_7195
Processing:  multicardioner_test+bg_7196
Processing:  multicardioner_test+bg_7197
Processing:  multicardioner_test+bg_7198
Processing:  multicardioner_test+bg_7199
Processing:  multi

Processing:  multicardioner_test+bg_7360
Processing:  multicardioner_test+bg_7361
Processing:  multicardioner_test+bg_7362
Processing:  multicardioner_test+bg_7363
Processing:  multicardioner_test+bg_7364
Processing:  multicardioner_test+bg_7365
Processing:  multicardioner_test+bg_7366
Processing:  multicardioner_test+bg_7367
Processing:  multicardioner_test+bg_7368
Processing:  multicardioner_test+bg_7369
Processing:  multicardioner_test+bg_737
Processing:  multicardioner_test+bg_7370
Processing:  multicardioner_test+bg_7371
Processing:  multicardioner_test+bg_7372
Processing:  multicardioner_test+bg_7373
Processing:  multicardioner_test+bg_7374
Processing:  multicardioner_test+bg_7375
Processing:  multicardioner_test+bg_7376
Processing:  multicardioner_test+bg_7377
Processing:  multicardioner_test+bg_7378
Processing:  multicardioner_test+bg_7379
Processing:  multicardioner_test+bg_738
Processing:  multicardioner_test+bg_7380
Processing:  multicardioner_test+bg_7381
Processing:  multi

Processing:  multicardioner_test+bg_7541
Processing:  multicardioner_test+bg_7542
Processing:  multicardioner_test+bg_7543
Processing:  multicardioner_test+bg_7544
Processing:  multicardioner_test+bg_7545
Processing:  multicardioner_test+bg_7546
Processing:  multicardioner_test+bg_7547
Processing:  multicardioner_test+bg_7548
Processing:  multicardioner_test+bg_7549
Processing:  multicardioner_test+bg_755
Processing:  multicardioner_test+bg_7550
Processing:  multicardioner_test+bg_7551
Processing:  multicardioner_test+bg_7552
Processing:  multicardioner_test+bg_7553
Processing:  multicardioner_test+bg_7554
Processing:  multicardioner_test+bg_7555
Processing:  multicardioner_test+bg_7556
Processing:  multicardioner_test+bg_7557
Processing:  multicardioner_test+bg_7558
Processing:  multicardioner_test+bg_7559
Processing:  multicardioner_test+bg_756
Processing:  multicardioner_test+bg_7560
Processing:  multicardioner_test+bg_7561
Processing:  multicardioner_test+bg_7562
Processing:  multi

Processing:  multicardioner_test+bg_7722
Processing:  multicardioner_test+bg_7723
Processing:  multicardioner_test+bg_7724
Processing:  multicardioner_test+bg_7725
Processing:  multicardioner_test+bg_7726
Processing:  multicardioner_test+bg_7727
Processing:  multicardioner_test+bg_7728
Processing:  multicardioner_test+bg_7729
Processing:  multicardioner_test+bg_773
Processing:  multicardioner_test+bg_7730
Processing:  multicardioner_test+bg_7731
Processing:  multicardioner_test+bg_7732
Processing:  multicardioner_test+bg_7733
Processing:  multicardioner_test+bg_7734
Processing:  multicardioner_test+bg_7735
Processing:  multicardioner_test+bg_7736
Processing:  multicardioner_test+bg_7737
Processing:  multicardioner_test+bg_7738
Processing:  multicardioner_test+bg_7739
Processing:  multicardioner_test+bg_774
Processing:  multicardioner_test+bg_7740
Processing:  multicardioner_test+bg_7741
Processing:  multicardioner_test+bg_7742
Processing:  multicardioner_test+bg_7743
Processing:  multi

Processing:  multicardioner_test+bg_818
Processing:  multicardioner_test+bg_819
Processing:  multicardioner_test+bg_82
Processing:  multicardioner_test+bg_820
Processing:  multicardioner_test+bg_821
Processing:  multicardioner_test+bg_822
Processing:  multicardioner_test+bg_823
Processing:  multicardioner_test+bg_824
Processing:  multicardioner_test+bg_825
Processing:  multicardioner_test+bg_826
Processing:  multicardioner_test+bg_827
Processing:  multicardioner_test+bg_828
Processing:  multicardioner_test+bg_829
Processing:  multicardioner_test+bg_83
Processing:  multicardioner_test+bg_830
Processing:  multicardioner_test+bg_831
Processing:  multicardioner_test+bg_832
Processing:  multicardioner_test+bg_833
Processing:  multicardioner_test+bg_834
Processing:  multicardioner_test+bg_835
Processing:  multicardioner_test+bg_836
Processing:  multicardioner_test+bg_837
Processing:  multicardioner_test+bg_838
Processing:  multicardioner_test+bg_839
Processing:  multicardioner_test+bg_84
Pro

In [36]:
df_sentences = pd.DataFrame.from_records(sentences)
df_sentences.head()

,filename,batch_number,batch_start,text
0,multicardioner_test+bg_1,1,0,Setting: cure primarie (PC).
1,multicardioner_test+bg_1,2,29,Motivo della consultazione: donna di 26 anni c...
2,multicardioner_test+bg_1,3,139,Era stata visitata al centro di emergenza del ...
3,multicardioner_test+bg_1,4,214,Ha spiegato un dolore addominale di 2 settiman...
4,multicardioner_test+bg_1,5,349,Anamnesi clinica: storia personale di nessun i...


In [37]:
df_sentences.to_csv(f'{output_path}/sentences.tsv', sep='\t', index=False)